# Homework and bake-off: Sentiment analysis

In [1]:
__author__ = "Christopher Potts"
__version__ = "CS224u, Stanford, Spring 2021"

## Contents

1. [Overview](#Overview)
1. [Methodological note](#Methodological-note)
1. [Set-up](#Set-up)
1. [Train set](#Train-set)
1. [Dev sets](#Dev-sets)
1. [A softmax baseline](#A-softmax-baseline)
1. [RNNClassifier wrapper](#RNNClassifier-wrapper)
1. [Error analysis](#Error-analysis)
1. [Homework questions](#Homework-questions)
  1. [Token-level differences [1 point]](#Token-level-differences-[1-point])
  1. [Training on some of the bakeoff data [1 point]](#Training-on-some-of-the-bakeoff-data-[1-point])
  1. [A more powerful vector-averaging baseline [2 points]](#A-more-powerful-vector-averaging-baseline-[2-points])
  1. [BERT encoding [2 points]](#BERT-encoding-[2-points])
  1. [Your original system [3 points]](#Your-original-system-[3-points])
1. [Bakeoff [1 point]](#Bakeoff-[1-point])

## Overview

This homework and associated bakeoff are devoted to supervised sentiment analysis using the ternary (positive/negative/neutral) version of the Stanford Sentiment Treebank (SST-3) as well as a new dev/test dataset drawn from restaurant reviews. Our goal in introducing the new dataset is to push you to create a system that performs well in both the movie and restaurant domains.

The homework questions ask you to implement some baseline system, and the bakeoff challenge is to define a system that does well at both the SST-3 test set and the new restaurant test set. Both are ternary tasks, and our central bakeoff score is the mean of the macro-FI scores for the two datasets. This assigns equal weight to all classes and datasets regardless of size.

The SST-3 test set will be used for the bakeoff evaluation. This dataset is already publicly distributed, so we are counting on people not to cheat by developing their models on the test set. You must do all your development without using the test set at all, and then evaluate exactly once on the test set and turn in the results, with no further system tuning or additional runs. __Much of the scientific integrity of our field depends on people adhering to this honor code__. 

One of our goals for this homework and bakeoff is to encourage you to engage in __the basic development cycle for supervised models__, in which you

1. Design a new system. We recommend starting with something simple.
1. Use `sst.experiment` to evaluate your system, using random train/test splits initially.
1. If you have time, compare your system with others using `sst.compare_models` or `utils.mcnemar`. (For discussion, see [this notebook section](sst_02_hand_built_features.ipynb#Statistical-comparison-of-classifier-models).)
1. Return to step 1, or stop the cycle and conduct a more rigorous evaluation with hyperparameter tuning and assessment on the `dev` set.

[Error analysis](#Error-analysis) is one of the most important methods for steadily improving a system, as it facilitates a kind of human-powered hill-climbing on your ultimate objective. Often, it takes a careful human analyst just a few examples to spot a major pattern that can lead to a beneficial change to the feature representations.

## Methodological note

You don't have to use the experimental framework defined below (based on `sst`). The only constraint we need to place on your system is that it must have a `predict_one` method that can map directly from an example text to a prediction, and it must be able to make predictions without having any information beyond the text. (For example, it can't depend on knowing which task the text comes from.) See [the bakeoff section below](#Bakeoff-[1-point]) for examples of functions that conform to this specification.

## Set-up

See [the first notebook in this unit](sst_01_overview.ipynb#Set-up) for set-up instructions.

In [2]:
from collections import Counter
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
import torch.nn as nn

from torch_rnn_classifier import TorchRNNClassifier
from torch_tree_nn import TorchTreeNN
import sst
import utils

In [3]:
SST_HOME = os.path.join('data', 'sentiment')

## Train set

Our primary train set is the SST-3 train set:

In [4]:
sst_train = sst.train_reader(SST_HOME)

In [5]:
sst_train.shape[0]

8544

In [6]:
sst_train.head()

,example_id,sentence,label,is_subtree
0,00001-00001,The Rock is destined to be the 21st Century 's...,positive,0
71,00002-00001,The gorgeously elaborate continuation of `` Th...,positive,0
144,00003-00001,Singer\/composer Bryan Adams contributes a sle...,positive,0
221,00004-00001,You 'd think by now America would have had eno...,neutral,0
258,00005-00001,Yet the act is still charming here .,positive,0


This is the train set we will use for all the regular homework questions. You are welcome to bring in new datasets for your original system. You are also free to add `include_subtrees=True`. This is very likely to lead to better systems, but it substantially increases the overall size of the dataset (from 8,544 examples to 159,274), which will in turn substantially increase the time it takes to run experiments.

See [this notebook](sst_01_overview.ipynb) for additional details of this dataset.

## Dev sets

We have two development set. SST3-dev consists of sentences from movie reviews, just like SST-3 train:

In [7]:
sst_dev = sst.dev_reader(SST_HOME)

In [8]:
len(sst_dev)

1101

Our new bakeoff dev set consists of sentences from restaurant reviews:

In [9]:
bakeoff_dev = sst.bakeoff_dev_reader(SST_HOME)

In [10]:
bakeoff_dev.sample(3, random_state=1).to_dict(orient='records')

[{'example_id': 57,
  'sentence': 'I would recommend that you make reservations in advance.',
  'label': 'neutral',
  'is_subtree': 0},
 {'example_id': 590,
  'sentence': 'We were welcomed warmly.',
  'label': 'positive',
  'is_subtree': 0},
 {'example_id': 1968,
  'sentence': 'We have been to Oceanaire twice in the last 6 weeks.',
  'label': 'neutral',
  'is_subtree': 0}]

Here is the label distribution:

In [11]:
bakeoff_dev.label.value_counts()

neutral     1019
positive     777
negative     565
Name: label, dtype: int64

The label distribution for the corresponding test set is similar to this.

## A softmax baseline

This example is here mainly as a reminder of how to use our experimental framework with linear models:

In [12]:
def unigrams_phi(text):
    return Counter(text.split())

Thin wrapper around `LogisticRegression` for the sake of `sst.experiment`:

In [13]:
def fit_softmax_classifier(X, y):
    mod = LogisticRegression(
        fit_intercept=True,
        solver='liblinear',
        multi_class='ovr')
    mod.fit(X, y)
    return mod

The experimental run with some notes:

In [14]:
softmax_experiment = sst.experiment(
    sst.train_reader(SST_HOME),   # Train on any data you like except SST-3 test!
    unigrams_phi,                 # Free to write your own!
    fit_softmax_classifier,       # Free to write your own!
    assess_dataframes=[sst_dev, bakeoff_dev]) # Free to change this during development!

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.628     0.689     0.657       428
     neutral      0.343     0.153     0.211       229
    positive      0.629     0.750     0.684       444

    accuracy                          0.602      1101
   macro avg      0.533     0.531     0.518      1101
weighted avg      0.569     0.602     0.575      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.272     0.690     0.390       565
     neutral      0.429     0.113     0.179      1019
    positive      0.409     0.346     0.375       777

    accuracy                          0.328      2361
   macro avg      0.370     0.383     0.315      2361
weighted avg      0.385     0.328     0.294      2361

Mean of macro-F1 scores: 0.416


`softmax_experiment` contains a lot of information that you can use for error analysis; see [this section below](#Error-analysis) for starter code.

## RNNClassifier wrapper

This section illustrates how to use `sst.experiment` with `TorchRNNClassifier`.

To featurize examples for an RNN, we can just get the words in order, letting the model take care of mapping them into an embedding space.

In [15]:
def rnn_phi(text):
    return text.split()

The model wrapper gets the vocabulary using `sst.get_vocab`. If you want to use pretrained word representations in here, then you can have `fit_rnn_classifier` build that space too; see [this notebook section for details](sst_03_neural_networks.ipynb#Pretrained-embeddings). See also [torch_model_base.py](torch_model_base.py) for details on the many optimization parameters that `TorchRNNClassifier` accepts.

In [16]:
def fit_rnn_classifier(X, y):
    sst_glove_vocab = utils.get_vocab(X, mincount=2)
    mod = TorchRNNClassifier(
        sst_glove_vocab,
        early_stopping=True)
    mod.fit(X, y)
    return mod

In [17]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    rnn_experiment = sst.experiment(
        sst.train_reader(SST_HOME),
        rnn_phi,
        fit_rnn_classifier,
        vectorize=False,  # For deep learning, use `vectorize=False`.
        assess_dataframes=[sst_dev, bakeoff_dev])

Stopping after epoch 64. Validation score did not improve by tol=1e-05 for more than 10 epochs. Final error is 0.1926556695252657

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.601     0.577     0.589       428
     neutral      0.332     0.336     0.334       229
    positive      0.603     0.622     0.612       444

    accuracy                          0.545      1101
   macro avg      0.512     0.512     0.512      1101
weighted avg      0.546     0.545     0.545      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.279     0.550     0.370       565
     neutral      0.417     0.091     0.150      1019
    positive      0.395     0.520     0.449       777

    accuracy                          0.342      2361
   macro avg      0.364     0.387     0.323      2361
weighted avg      0.377     0.342     0.301      2361

Mean of macro-F1 scores: 0.417


## Error analysis

This section begins to build an error-analysis framework using the dicts returned by `sst.experiment`. These have the following structure:

```
'model': trained model
'phi': the feature function used
'train_dataset':
   'X': feature matrix
   'y': list of labels
   'vectorizer': DictVectorizer,
   'raw_examples': list of raw inputs, before featurizing   
'assess_datasets': list of datasets, each with the same structure as the value of 'train_dataset'
'predictions': list of lists of predictions on the assessment datasets
'metric': `score_func.__name__`, where `score_func` is an `sst.experiment` argument
'score': the `score_func` score on the each of the assessment dataasets
```
The following function just finds mistakes, and returns a `pd.DataFrame` for easy subsequent processing:

In [23]:
def find_errors(experiment):
    """Find mistaken predictions.

    Parameters
    ----------
    experiment : dict
        As returned by `sst.experiment`.

    Returns
    -------
    pd.DataFrame

    """
    dfs = []
    for i, dataset in enumerate(experiment['assess_datasets']):
        df = pd.DataFrame({
            'raw_examples': dataset['raw_examples'],
            'predicted': experiment['predictions'][i],
            'gold': dataset['y']})
        df['correct'] = df['predicted'] == df['gold']
        df['dataset'] = i
        dfs.append(df)
    return pd.concat(dfs)

In [24]:
# softmax_analysis = find_errors(softmax_experiment)

In [25]:
# rnn_analysis = find_errors(rnn_experiment)

Here we merge the sotmax and RNN experiments into a single DataFrame:

In [26]:
# analysis = softmax_analysis.merge(
#     rnn_analysis, left_on='raw_examples', right_on='raw_examples')

# analysis = analysis.drop('gold_y', axis=1).rename(columns={'gold_x': 'gold'})

The following code collects a specific subset of examples; small modifications to its structure will give you different interesting subsets:

In [27]:
# # Examples where the softmax model is correct, the RNN is not,
# # and the gold label is 'positive'

# error_group = analysis[
#     (analysis['predicted_x'] == analysis['gold'])
#     &
#     (analysis['predicted_y'] != analysis['gold'])
#     &
#     (analysis['gold'] == 'positive')
# ]

In [28]:
# error_group.shape[0]

In [29]:
# for ex in error_group['raw_examples'].sample(5, random_state=1):
#     print("="*70)
#     print(ex)

## Homework questions

Please embed your homework responses in this notebook, and do not delete any cells from the notebook. (You are free to add as many cells as you like as part of your responses.)

### Token-level differences [1 point]

We can begin to get a sense for how our two dev sets differ by considering the most frequent tokens from each. This question asks you to begin such analysis.

Your task: write a function `get_token_counts` that, given a `pd.DataFrame` in the format of our datasets, tokenizes the example sentences based on whitespace and creates a count distribution over all of the tokens. The function should return a `pd.Series` sorted by frequency; if you create a count dictionary `d`, then `pd.Series(d).sort_values(ascending=False)` will give you what you need.

In [30]:
def get_token_counts(df):
    ##### YOUR CODE HERE
    counts = Counter()
    for sent in df['sentence']:
        counts += unigrams_phi(sent)
    return pd.Series(counts).sort_values(ascending=False)

In [31]:
def test_get_token_counts(func):
    df = pd.DataFrame([
        {'sentence': 'a a b'},
        {'sentence': 'a b a'},
        {'sentence': 'a a a b.'}])
    result = func(df)
    for token, expected in (('a', 7), ('b', 2), ('b.', 1)):
        actual = result.loc[token]
        assert actual == expected, \
            "For token {}, expected {}; got {}".format(
            token, expected, actual)

In [32]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_get_token_counts(get_token_counts)

As you develop your original system, you might review these results. The two dev sets have different vocabularies and different low-level encoding details that are sure to impact model performance, especially when one considers that the train set is like `sst_dev` in all these respects. For additional discussion, see [this notebook section](sst_01_overview.ipynb#Tokenization).

### Training on some of the bakeoff data [1 point]

We have so far presented the bakeoff dev set as purely for evaluation. Since the train set consists entirely of SST-3 data, this makes the bakeoff split especially challenging. We might be able to reduce the challenging by adding some of the bakeoff dev set to the train set, keeping some of it for evaluation. The current question asks to begin explore the effects of such training.

Your task: write a function `run_mixed_training_experiment`. The function should:

1. Take as inputs (a) a model training wrapper like `fit_softmax_classifier` and (b) an integer `bakeoff_train_size` specifying the number of examples from `bakeoff_dev` that should be included in the train set.
1. Split `bakeoff_dev` so that the first `bakeoff_train_size` examples are in the train set and the rest are used for evaluation.
1. Use `sst.experiment` with the user-supplied model training wrapper, `unigram_phi` as defined above, and a train set that consists of SST-3 train and the train portion of `bakeoff_dev` as defined in step 2. The value of `assess_dataframes` should be a list consisting of the SST-3 dev set and the evaluation portion of `bakeoff_dev` as defined in step 2.
1. Return the return value of `sst.experiment`.

The function `test_run_mixed_training_experiment` will help you iterate to the required design.

In [33]:
def run_mixed_training_experiment(wrapper_func, bakeoff_train_size):
    ##### YOUR CODE HERE
    
    return sst.experiment(
        [sst.train_reader(SST_HOME), bakeoff_dev[:bakeoff_train_size]],
        unigrams_phi,
        wrapper_func,
        assess_dataframes=[sst_dev, bakeoff_dev[bakeoff_train_size:]])


In [34]:
def test_run_mixed_training_experiment(func):
    bakeoff_train_size = 1000
    experiment = func(fit_softmax_classifier, bakeoff_train_size)

    assess_size = len(experiment['assess_datasets'])
    assert len(experiment['assess_datasets']) == 2, \
        ("The evaluation should be done on two datasets: "
         "SST3 and part of the bakeoff dev set. "
         "You have {} datasets.".format(assess_size))

    bakeoff_test_size = bakeoff_dev.shape[0] - bakeoff_train_size
    expected_eval_examples = bakeoff_test_size + sst_dev.shape[0]
    eval_examples = sum(len(d['raw_examples']) for d in experiment['assess_datasets'])
    assert expected_eval_examples == eval_examples, \
        "Expected {} evaluation examples; got {}".format(
        expected_eval_examples, eval_examples)

In [35]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_run_mixed_training_experiment(run_mixed_training_experiment)

Assessment dataset 1
              precision    recall  f1-score   support

    negative      0.627     0.671     0.648       428
     neutral      0.319     0.162     0.214       229
    positive      0.638     0.757     0.692       444

    accuracy                          0.599      1101
   macro avg      0.528     0.530     0.518      1101
weighted avg      0.567     0.599     0.576      1101

Assessment dataset 2
              precision    recall  f1-score   support

    negative      0.471     0.412     0.440       320
     neutral      0.588     0.590     0.589       612
    positive      0.503     0.548     0.525       429

    accuracy                          0.535      1361
   macro avg      0.521     0.517     0.518      1361
weighted avg      0.534     0.535     0.534      1361

Mean of macro-F1 scores: 0.518


### A more powerful vector-averaging baseline [2 points]

In [Distributed representations as features](sst_03_neural_networks.ipynb#Distributed-representations-as-features), we looked at a baseline for the ternary SST-3 problem in which each example is modeled as the mean of its GloVe representations. A `LogisticRegression` model was used for prediction. A neural network might do better with these representations, since there might be complex relationships between the input feature dimensions that a linear classifier can't learn. To address this question, we want to get set up to run the experiment with a shallow neural classifier. 

Your task: write and submit a model wrapper function around `TorchShallowNeuralClassifier`. This function should implement hyperparameter search according to this specification:

* Set `early_stopping=True` for all experiments.
* Using 3-fold cross-validation, exhaustively explore this set of hyperparameter combinations:
  * The hidden dimensionality at 50, 100, and 200.
  * The hidden activation function as `nn.Tanh()` and `nn.ReLU()`.
* For all other parameters to `TorchShallowNeuralClassifier`, use the defaults.

See [this notebook section](sst_02_hand_built_features.ipynb#Hyperparameter-search) for examples. You are not required to run a full evaluation with this function using `sst.experiment`, but we assume you will want to.

We're not evaluating the quality of your model. (We've specified the protocols completely, but there will still be variation in the results.) However, the primary goal of this question is to get you thinking more about this strong baseline feature representation scheme for SST-3, so we're sort of hoping you feel compelled to try out variations on your own.

In [36]:
# DATA_HOME = 'data'

# GLOVE_HOME = os.path.join(DATA_HOME, 'glove.6B')

# glove_lookup = utils.glove2dict(
#     os.path.join(GLOVE_HOME, 'glove.6B.300d.txt'))

# def vsm_phi(text, lookup, np_func=np.mean):
#     """Represent `tree` as a combination of the vector of its words.

#     Parameters
#     ----------
#     text : str

#     lookup : dict
#         From words to vectors.

#     np_func : function (default: np.sum)
#         A numpy matrix operation that can be applied columnwise,
#         like `np.mean`, `np.sum`, or `np.prod`. The requirement is that
#         the function take `axis=0` as one of its arguments (to ensure
#         columnwise combination) and that it return a vector of a
#         fixed length, no matter what the size of the tree is.

#     Returns
#     -------
#     np.array, dimension `X.shape[1]`

#     """
#     allvecs = np.array([lookup[w] for w in text.split() if w in lookup])
#     if len(allvecs) == 0:
#         dim = len(next(iter(lookup.values())))
#         feats = np.zeros(dim)
#     else:
#         feats = np_func(allvecs, axis=0)
#     return feats

# def glove_phi(text, np_func=np.mean):
#     return vsm_phi(text, glove_lookup, np_func=np_func)

In [37]:
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier

def fit_shallow_neural_classifier_with_hyperparameter_search(X, y):
    ##### YOUR CODE HERE
    basemod = TorchShallowNeuralClassifier(early_stopping=True)
    cv = 3
    param_grid = {
        'hidden_dim': [50, 100, 200],
        'hidden_activation': [nn.Tanh(), nn.ReLU()]}
    bestmod = utils.fit_classifier_with_hyperparameter_search(
        X, y, basemod, cv, param_grid)
    return bestmod

In [38]:
# shallow_neural_experiment = sst.experiment(
#     sst.train_reader(SST_HOME),
#     glove_phi,
#     fit_shallow_neural_classifier_with_hyperparameter_search,
#     vectorize=False,
#     assess_dataframes=sst_dev) # use only `sst_dev` to directly compare with naive mean

### BERT encoding [2 points]

We might hypothesize that encoding our examples with BERT will yield improvements over the GloVe averaging method explored in the previous question, since BERT implements a much more complex and data-driven function for this kind of combination. This question asks you to begin exploring this general hypothesis.

Your task: write a function `hf_cls_phi` that uses Hugging Face functionality to encode individual examples with BERT and returns the final output representation above the [CLS] token.

You are not required to evaluate this feature function, but it is easy to do so with `sst.experiment` and `vectorize=False` (since your feature function directly encodes every example as a vector). Your code should also be a natural basis for even more powerful approaches – for example, it might be even better to pool all the output states rather than using just the first output state. Another option is [fine-tuning](finetuning.ipynb).

In [39]:
from transformers import BertModel, BertTokenizer
import vsm

# Instantiate a Bert model and tokenizer based on `bert_weights_name`:
bert_weights_name = 'bert-base-uncased'
##### YOUR CODE HERE
bert_tokenizer = BertTokenizer.from_pretrained(bert_weights_name)
bert_model = BertModel.from_pretrained(bert_weights_name)


def hf_cls_phi(text):
    # Get the ids. `vsm.hf_encode` will help; be sure to
    # set `add_special_tokens=True`.
    ##### YOUR CODE HERE
    ids = vsm.hf_encode(text, bert_tokenizer, add_special_tokens=True)

    # Get the BERT representations. `vsm.hf_represent` will help:
    ##### YOUR CODE HERE
    reps = vsm.hf_represent(ids, bert_model)


    # Index into `reps` to get the representation above [CLS].
    # The shape of `reps` should be (1, n, 768), where n is the
    # number of tokens. You need the 0th element of the 2nd dim:
    ##### YOUR CODE HERE
    cls_rep = reps[0, 0, :]


    # These conversions should ensure that you can work with the
    # representations flexibly. Feel free to change the variable
    # name:
    return cls_rep.cpu().numpy()

In [40]:
def test_hf_cls_phi(func):
    rep = func("Just testing!")

    expected_shape = (768,)
    result_shape = rep.shape
    assert rep.shape == (768,), \
        "Expected shape {}; got {}".format(
        expected_shape, result_shape)

    # String conversion to avoid precision errors:
    expected_first_val = str(0.1709)
    result_first_val = "{0:.04f}".format(rep[0])

    assert expected_first_val == result_first_val, \
        ("Unexpected representation values. Expected the "
        "first value to be {}; got {}".format(
            expected_first_val, result_first_val))

In [41]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
    test_hf_cls_phi(hf_cls_phi)

Note: encoding all of SST-3 train (no subtrees) takes about 11 minutes on my 2015 iMac, CPU only (32GB).

### Your original system [3 points]

Your task is to develop an original model for the SST-3 problem and our new bakeoff dataset. There are many options. If you spend more than a few hours on this homework problem, you should consider letting it grow into your final project! Here are some relatively manageable ideas that you might try:

1. We didn't systematically evaluate the `bidirectional` option to the `TorchRNNClassifier`. Similarly, that model could be tweaked to allow multiple LSTM layers (at present there is only one), and you could try adding layers to the classifier portion of the model as well.

1. We've already glimpsed the power of rich initial word representations, and later in the course we'll see that smart initialization usually leads to a performance gain in NLP, so you could perhaps achieve a winning entry with a simple model that starts in a great place.

1. Our [practical introduction to contextual word representations](finetuning.ipynb) covers pretrained representations and interfaces that are likely to boost the performance of any system.

We want to emphasize that this needs to be an __original__ system. It doesn't suffice to download code from the Web, retrain, and submit. You can build on others' code, but you have to do something new and meaningful with it. See the course website for additional guidance on how original systems will be evaluated.

In the cell below, please provide a brief technical description of your original system, so that the teaching team can gain an understanding of what it does. This will help us to understand your code and analyze all the submissions to identify patterns and strategies.  We also ask that you report the best score your system got during development (your best average of macro-F1 scores), just to help us understand how systems performed overall.

In [42]:
# PLEASE MAKE SURE TO INCLUDE THE FOLLOWING BETWEEN THE START AND STOP COMMENTS:
#   1) Textual description of your system.
#   2) The code for your original system.
#   3) The score achieved by your system in place of MY_NUMBER.
#        With no other changes to that line.
#        You should report your score as a decimal value <=1.0
# PLEASE MAKE SURE NOT TO DELETE OR EDIT THE START AND STOP COMMENTS

# NOTE: MODULES, CODE AND DATASETS REQUIRED FOR YOUR ORIGINAL SYSTEM
# SHOULD BE ADDED BELOW THE 'IS_GRADESCOPE_ENV' CHECK CONDITION. DOING
# SO ABOVE THE CHECK MAY CAUSE THE AUTOGRADER TO FAIL.

# START COMMENT: Enter your system description in this cell.
# My peak score was: 0.534
# 
# Description: 
# 
# Since we are trying to do fast domain adaptation, 
# we implemented a prototypical network (Snell et al., 2017)
# with DistillBert's [CLS] rep as the initialized encoder. 
# 
# Prototypical networks is a meta-learning framework popular in few-shot learning
# as a way to achieve fast domain adaptation, and have performed
# extremely well on certain few-shot learning benchmarks, such as Omniglot.
#
# During metatrain, we train the model to do 3-shot (per class) classification.
# During metadev, we ask the model to perform 3-shot classification on unseen domains.
# During metatest, we increase the k in k-shot, as in general it is easy to
# collect more than 3 examples per class available even for a new domain.
#
# For our meta-training and meta-dev data, we used sentiment analysis
# datasets from the following domains:
# - Amazon reviews by category (https://nijianmo.github.io/amazon/index.html#subsets)
# - DynaSent Round 1 Yelp Dataset
# - Airline reviews on Twitter (https://www.kaggle.com/crowdflower/twitter-airline-sentiment/)
# - sst_train
# These give us a total of 27 domains, and we hold out 5 domains for meta-val.
# 
# For meta-test, we use sst_dev and bakeoff_dev.
#
# Given the training and eval paradigm differ so drastically from standard supervised learning,
# we had to leverage custom Trainer and Model implementations, but HuggingFace made it doable.
# 
# Analysis:
# 
# At meta-val time, we eval'ed our model's ability to perform 3-shot classification
# on a set of 5 held-out tasks, and the model was able to obtain an F1 score of 0.642.
# However, at (meta)test time, which uses our dev sets, the score had a huge drop.
# Upon closer inspection, since most of our task-specific datasets come from Amazon,
# and all held-out tasks happen to be Amazon subsets, we observe meta-overfitting:
# overfitting to the overwhelming presence of Amazon reviews among our tasks,
# similar to how a traditional classifier might favor the dominant classes.
# 
# To remedy this issue, we could potentially explore upsampling certain domains/tasks,
# or including more diverse datasets.
#
# The full colab development is here:
#     https://colab.research.google.com/drive/1C9lzeAeL2W7ofjZz9nWRwyJdQxkhbKig?usp=sharing


if 'IS_GRADESCOPE_ENV' not in os.environ:
    from typing import List, Dict, Optional
    import random, json
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report

    import torch
    import torch.nn as nn
    import transformers
    import vsm
    
    from torch.utils.data import Dataset
    
    from transformers import AutoTokenizer, AutoModel
    from transformers import Trainer
    from transformers.trainer import *
    from transformers.modeling_outputs import SequenceClassifierOutput
    
    class ProtoNet(nn.Module):
        def __init__(self, model_name): # TODO: add device info potentially
            super(ProtoNet, self).__init__()
            self.encoder = AutoModel.from_pretrained(model_name)
            self.num_classes = 3

        def _encode(self, ids):
            return self.encoder(**ids, output_hidden_states=True).hidden_states[-1][:, 0, :]

        def encode(self, ids, is_support):
            if is_support:
                reps_by_task = [self._encode(ids_per_class) for ids_per_class in ids] # list of len N, each elt is a (K,D) tensor
                reps = torch.vstack([rep.unsqueeze(0) for rep in reps_by_task]) # (N, K, D)
            else:
                reps = self._encode(ids) # (N*Q, D)
            return reps  


        @staticmethod
        def euclidean_dist(centroids, queries):
            def calc_norm_squared(tensor):
                return torch.mul(tensor, tensor).sum(dim=-1)

            centroids_norm_squared = calc_norm_squared(centroids).unsqueeze(0) # (1, N)
            queries_norm_squared = calc_norm_squared(queries).unsqueeze(-1) # (N * Q, 1)
            dot_prod = torch.matmul(queries, torch.transpose(centroids, 0, 1)) # (N*Q, N)
            distances = queries_norm_squared - 2 * dot_prod + centroids_norm_squared

            return distances


        def forward(self, supports, queries): #, query_labels):
            '''
              Args:
                supports: support examples, List[List[str]] of shape [N, K]
                queries: query examples, List[str] of length N * Q
                # query_labels: one-hot encodings of the labels of the queries with shape N * Q, N

                N: number of classes, 3 in this case
                K: K in K-shot learning, i.e., number of support examples per class
                Q: number of query points per class

              Returns:
                Logits
            '''

            # get encoder reps
            support_reps = self.encode(supports, is_support=True) # (N, K, D)
            query_reps = self.encode(queries, is_support=False) # (N * Q, D)

            # get centroids
            centroids = torch.mean(support_reps, dim=1) # (N, D)

            # calculate euclidean distance
            distances = self.euclidean_dist(centroids, query_reps)
            logits = -distances # (N*Q, N)
            return SequenceClassifierOutput(logits=logits)



    class ProtoNetTrainer(Trainer):
        def compute_loss(self, model, inputs, return_outputs=False):
            labels = inputs["labels"]
            outputs = model(inputs["supports"], inputs["queries"])
            logits = outputs.logits
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)
            return (loss, outputs) if return_outputs else loss

        def prediction_loop(
            self,
            dataloader,
            description: str,
            prediction_loss_only: Optional[bool] = None,
            ignore_keys: Optional[List[str]] = None,
            metric_key_prefix: str = "eval"):
            """
            Prediction/evaluation loop, shared by :obj:`Trainer.evaluate()` and :obj:`Trainer.predict()`.
            Works both with or without labels.

            Note: This is almost identical to source code, with a 1-line change in `num_examples`.
            This has to be in place given the quirks of ProtoNet.
            """
            if not isinstance(dataloader.dataset, collections.abc.Sized):
                raise ValueError("dataset must implement __len__")
            prediction_loss_only = (
                prediction_loss_only if prediction_loss_only is not None else self.args.prediction_loss_only
            )

            if self.args.deepspeed and not self.args.do_train:
                # no harm, but flagging to the user that deepspeed config is ignored for eval
                # flagging only for when --do_train wasn't passed as only then it's redundant
                logger.info("Detected the deepspeed argument but it will not be used for evaluation")

            model = self._wrap_model(self.model, training=False)

            # if full fp16 is wanted on eval and this ``evaluation`` or ``predict`` isn't called while
            # ``train`` is running, half it first and then put on device
            if not self.is_in_train and self.args.fp16_full_eval:
                model = model.half().to(self.args.device)

            batch_size = dataloader.batch_size
            num_examples = self.num_examples(dataloader) * self.model.num_classes
            logger.info(f"***** Running {description} *****")
            logger.info(f"  Num examples = {num_examples}")
            logger.info(f"  Batch size = {batch_size}")
            losses_host: torch.Tensor = None
            preds_host: Union[torch.Tensor, List[torch.Tensor]] = None
            labels_host: Union[torch.Tensor, List[torch.Tensor]] = None

            world_size = max(1, self.args.world_size)

            eval_losses_gatherer = DistributedTensorGatherer(world_size, num_examples, make_multiple_of=batch_size)
            if not prediction_loss_only:
                # The actual number of eval_sample can be greater than num_examples in distributed settings (when we pass
                # a batch size to the sampler)
                make_multiple_of = None
                if hasattr(dataloader, "sampler") and isinstance(dataloader.sampler, SequentialDistributedSampler):
                    make_multiple_of = dataloader.sampler.batch_size
                preds_gatherer = DistributedTensorGatherer(world_size, num_examples, make_multiple_of=make_multiple_of)
                labels_gatherer = DistributedTensorGatherer(world_size, num_examples, make_multiple_of=make_multiple_of)

            model.eval()

            if is_torch_tpu_available():
                dataloader = pl.ParallelLoader(dataloader, [self.args.device]).per_device_loader(self.args.device)

            if self.args.past_index >= 0:
                self._past = None

            self.callback_handler.eval_dataloader = dataloader

            for step, inputs in enumerate(dataloader):
                loss, logits, labels = self.prediction_step(model, inputs, prediction_loss_only, ignore_keys=ignore_keys)
                if loss is not None:
                    losses = loss.repeat(batch_size)
                    losses_host = losses if losses_host is None else torch.cat((losses_host, losses), dim=0)
                if logits is not None:
                    preds_host = logits if preds_host is None else nested_concat(preds_host, logits, padding_index=-100)
                if labels is not None:
                    labels_host = labels if labels_host is None else nested_concat(labels_host, labels, padding_index=-100)
                self.control = self.callback_handler.on_prediction_step(self.args, self.state, self.control)

                # Gather all tensors and put them back on the CPU if we have done enough accumulation steps.
                if self.args.eval_accumulation_steps is not None and (step + 1) % self.args.eval_accumulation_steps == 0:
                    eval_losses_gatherer.add_arrays(self._gather_and_numpify(losses_host, "eval_losses"))
                    if not prediction_loss_only:
                        preds_gatherer.add_arrays(self._gather_and_numpify(preds_host, "eval_preds"))
                        labels_gatherer.add_arrays(self._gather_and_numpify(labels_host, "eval_label_ids"))

                    # Set back to None to begin a new accumulation
                    losses_host, preds_host, labels_host = None, None, None

            if self.args.past_index and hasattr(self, "_past"):
                # Clean the state at the end of the evaluation loop
                delattr(self, "_past")

            # Gather all remaining tensors and put them back on the CPU
            eval_losses_gatherer.add_arrays(self._gather_and_numpify(losses_host, "eval_losses"))
            if not prediction_loss_only:
                preds_gatherer.add_arrays(self._gather_and_numpify(preds_host, "eval_preds"))
                labels_gatherer.add_arrays(self._gather_and_numpify(labels_host, "eval_label_ids"))

            eval_loss = eval_losses_gatherer.finalize()
            preds = preds_gatherer.finalize() if not prediction_loss_only else None
            label_ids = labels_gatherer.finalize() if not prediction_loss_only else None

            if self.compute_metrics is not None and preds is not None and label_ids is not None:
                metrics = self.compute_metrics(EvalPrediction(predictions=preds, label_ids=label_ids))
            else:
                metrics = {}

            # To be JSON-serializable, we need to remove numpy types or zero-d tensors
            metrics = denumpify_detensorize(metrics)

            if eval_loss is not None:
                metrics[f"{metric_key_prefix}_loss"] = eval_loss.mean().item()

            # Prefix all keys with metric_key_prefix + '_'
            for key in list(metrics.keys()):
                if not key.startswith(f"{metric_key_prefix}_"):
                    metrics[f"{metric_key_prefix}_{key}"] = metrics.pop(key)

            return PredictionOutput(predictions=preds, label_ids=label_ids, metrics=metrics)


    class DataGenerator:
        def __init__(self, task_datasets, num_support, num_query, model_name):
            self.num_support = num_support
            self.num_query = num_query
            self.datasets = task_datasets

            self.classes = ['negative', 'neutral', 'positive']
            self.cls_idx_map = {cls: idx for idx, cls in enumerate(self.classes)}

            self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        def _tokenize(self, texts: List[str]):
            return self.tokenizer(texts, padding='max_length', truncation=True, return_tensors="pt")

        def tokenize(self, texts, is_support):
            if is_support:
                return [self._tokenize(texts_by_class) for texts_by_class in texts]
            else:
                return self._tokenize(texts)

        def sample_batch(self, task=None):
            if task is None:
                task, task_ds = random.choice(list(self.datasets.items()))
            else:
                task_ds = self.datasets[task]
            support = task_ds['support']
            query = task_ds['query']

            batch_support = [random.sample(support[cls], self.num_support) for cls in self.classes]

            batch_query = [random.sample(query[cls], self.num_query) for cls in self.classes]
            batch_query = [q for query in batch_query for q in query] # flatten
            query_labels = [self.cls_idx_map[cls] for cls in self.classes for _ in range(self.num_query)]

            randomize_query = list(range(len(query_labels)))
            random.shuffle(randomize_query)
            batch_query = [batch_query[i] for i in randomize_query]
            query_labels = [query_labels[i] for i in randomize_query]

            return {'supports': self.tokenize(batch_support, True), 
                    'queries': self.tokenize(batch_query, False),
                    'labels': torch.tensor(query_labels, dtype=torch.long)}


    class MetaTrainDataset(Dataset):
        def __init__(self, task_datasets, num_support, num_query, model_name, num_steps=10000):
            self.data_generator = DataGenerator(task_datasets, num_support, num_query, model_name)
            self.num_steps = num_steps
            self.examples = [self.data_generator.sample_batch() for _ in range(self.num_steps)]

        def __len__(self):
            return self.num_steps

        def __getitem__(self, idx):
            return self.examples[idx]

    class MetaValDataset(Dataset):
        def __init__(self, task_datasets, num_support, num_query, model_name, num_examples_per_task=5):
            self.datasets = task_datasets
            self.data_generator = DataGenerator(task_datasets, num_support, num_query, model_name)
            self.examples = [self.data_generator.sample_batch(task) for task in self.datasets.keys()
                                 for _ in range(num_examples_per_task)]

        def __len__(self):
            return len(self.examples)

        def __getitem__(self, idx):
            return self.examples[idx]

    def pass_data_collator(features) -> Dict[str, torch.Tensor]:
        return features[0]
    
    model_name = 'distilbert-base-uncased'

    def construct_meta(df, sentence_name='sentence', label_name='label'):
        keys = ['negative', 'neutral', 'positive']
        sentences = df[sentence_name].tolist()
        labels = df[label_name].tolist()
        support_sentences, query_sentences, support_labels, query_labels = train_test_split(sentences, labels, train_size=0.8)

        support = {key: [] for key in keys}
        for sentence, label in zip(support_sentences, support_labels):
            support[label].append(sentence)

        query = {key: [] for key in keys}
        for sentence, label in zip(query_sentences, query_labels):
            query[label].append(sentence)

        return {'support': support, 'query': query} 
    
    task_datasets = {}

    # amazon
    amazon_data_dir = os.path.join(SST_HOME, 'amazon_by_category')
    for fp in os.listdir(amazon_data_dir):
        task_name = f"amz_{fp.split('.')[0]}"
        df = pd.read_json(os.path.join(amazon_data_dir, fp))
        if len(df) > 3000:
            task_datasets[task_name] = construct_meta(df) 
    
    # sst
    task_datasets['sst_train'] = construct_meta(sst_train)

    # dynasent
    dynasent_data_dir = os.path.join(SST_HOME, 'dynasent-v1.1')
    dynasent_train_filename = os.path.join(dynasent_data_dir, 'dynasent-v1.1-round01-yelp-train.jsonl')
    dynasent_dev_filename = os.path.join(dynasent_data_dir, 'dynasent-v1.1-round01-yelp-dev.jsonl')

    def load_dataset(src_filenames, labels=None):
        data = []
        for filename in src_filenames:
            with open(filename) as f:
                for line in f:
                    d = json.loads(line)
                    if labels is None or d['gold_label'] in labels:
                        data.append(d)
        return data


    ternary_labels = ('positive', 'negative', 'neutral')
    dynasent = load_dataset([dynasent_train_filename, dynasent_dev_filename], labels=ternary_labels)
    dynasent_df = pd.DataFrame.from_dict(dynasent).sample(10000, random_state=42)

    task_datasets['dynasent'] = construct_meta(dynasent_df, label_name='gold_label')

    # airline reviews
    airline_tweets_df = pd.read_csv(os.path.join(SST_HOME, 'twitter_airline/Tweets.csv'))
    airline_tweets_df_filtered = airline_tweets_df[airline_tweets_df.airline_sentiment_confidence > 0.7]

    task_datasets['airline'] = construct_meta(airline_tweets_df_filtered, sentence_name='text', label_name='airline_sentiment')
    
    random.seed(42)
    metaval_tasks = random.sample(task_datasets.keys(), int(len(task_datasets) * 0.2))

    metatrain_task_datasets = {task: ds for task, ds in task_datasets.items() if task not in metaval_tasks}
    metaval_task_datasets = {task: ds for task, ds in task_datasets.items() if task in metaval_tasks}

    metatrain_dataset = MetaTrainDataset(metatrain_task_datasets, 3 , 1, model_name, num_steps=3000)

    metaval_dataset = MetaValDataset(metaval_task_datasets, 3, 1, model_name)

    trainer_args = transformers.TrainingArguments(output_dir='./experiments', 
                                              per_device_eval_batch_size=1, 
                                              per_device_train_batch_size=1,
                                              logging_steps=100,
                                              load_best_model_at_end=True,
                                              metric_for_best_model='f1-score')

    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)
        target_names = ['negative', 'neutral', 'positive']
        return classification_report(labels, preds, target_names=target_names, output_dict=True)['macro avg']

    protonet = ProtoNet(model_name)
    trainer = ProtoNetTrainer(model=protonet, 
                              args=trainer_args, 
                              data_collator=pass_data_collator, 
                              train_dataset=train_dataset,
                              compute_metrics=compute_metrics)
    # trainer.train()

    # ======== EVAL =======
    metatest_task_dataset = construct_metatest(pd.concat([sst_dev, bakeoff_dev]).sample(frac=1))
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    def tokenize(texts, is_support):
        def _tokenize(texts: List[str]):
            return tokenizer(texts, padding='max_length', truncation=True, return_tensors="pt")

        if is_support:
            return [_tokenize(texts_by_class) for texts_by_class in texts]
        else:
            return _tokenize(texts)
        
    def calculate_prototypes(support_set, device='cuda:0'):
        support_ids = tokenize(support_set, is_support=True)
        def move_dict(dict_tensors):
            return {key: val.to(device) for key, val in dict_tensors.items()}
        with torch.no_grad():
            reps = protonet.encode([move_dict(support_ids_per_class) for support_ids_per_class in support_ids], is_support=True)
        return torch.mean(reps, dim=1).cpu().numpy()
    
    prototypes = calculate_prototypes(metatest_task_dataset['support'])

    def euclidean_dist_np(centroids, query):
        def calc_norm_squared(arr):
            return np.multiply(arr, arr).sum(axis=-1)
    
        centroids_norm_squared = calc_norm_squared(centroids).reshape(1, -1) # (1, N)
        queries_norm_squared = calc_norm_squared(query).reshape(-1, 1) # (N * Q, 1)
        dot_prod = np.matmul(query, centroids.T) # (N*Q, N)
        distances = queries_norm_squared - 2 * dot_prod + centroids_norm_squared
        return distances


    def predict_one(text):
        labels = ['negative', 'neutral', 'positive']
        ids = tokenize(text, is_support=False)
        def move_dict(dict_tensors, device='cuda:0'):
            return {key: val.to(device) for key, val in dict_tensors.items()}
        with torch.no_grad():
            reps = protonet.encode(move_dict(ids), is_support=False).cpu().numpy()
        distances = euclidean_dist_np(prototypes, reps)
        label_id = int(np.argmin(distances, -1))
        return labels[label_id]

    '''
    Dev eval from Colab:
    
    Assessment dataset 1
                  precision    recall  f1-score   support

        negative      0.803     0.514     0.627       428
         neutral      0.354     0.306     0.328       229
        positive      0.604     0.856     0.708       444

        accuracy                          0.609      1101
       macro avg      0.587     0.559     0.554      1101
    weighted avg      0.629     0.609     0.597      1101

    Assessment dataset 2
                  precision    recall  f1-score   support

        negative      0.525     0.391     0.448       565
         neutral      0.573     0.431     0.492      1019
        positive      0.498     0.753     0.600       777

        accuracy                          0.527      2361
       macro avg      0.532     0.525     0.513      2361
    weighted avg      0.537     0.527     0.517      2361

    Mean of macro-F1 scores: 0.534
    '''

                                    
                                    

# STOP COMMENT: Please do not remove this comment.

## Bakeoff [1 point]

As we said above, the bakeoff evaluation data is the official SST test-set release and a new test set derived from the same sources and labeling methods as for `bakeoff_dev`.

For this bakeoff, you'll evaluate your original system from the above homework problem on these test sets. Our metric will be the mean of the macro-F1 values, which weights both datasets equally despite their differing sizes.

The central requirement for your system is that you have define a `predict_one` method for it that maps a text (str) directly to a label prediction – one of 'positive', 'negative', 'neutral'. If you used `sst.experiment` with `vectorize=True`, then the following function (for `softmax_experiment`) will be easy to adapt – you probably just need to change the variable `softmax_experiment` to the variable for your experiment output.

In [43]:
def predict_one_softmax(text):
    # Singleton list of feature dicts:
    feats = [softmax_experiment['phi'](text)]
    # Vectorize to get a feature matrix:
    X = softmax_experiment['train_dataset']['vectorizer'].transform(feats)
    # Standard sklearn `predict` step:
    preds = softmax_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

If you used an RNN like the one we demoed above, then featurization is a bit more straightforward:

In [44]:
def predict_one_rnn(text):
    # List of tokenized examples:
    X = [rnn_experiment['phi'](text)]
    # Standard `predict` step on a list of lists of str:
    preds = rnn_experiment['model'].predict(X)
    # Be sure to return the only member of the predictions,
    # rather than the singleton list:
    return preds[0]

The following function is used to create the bakeoff submission file. Its arguments are your `predict_one` function and an output filename (str).

In [45]:
def create_bakeoff_submission(
        predict_one_func,
        output_filename='cs224u-sentiment-bakeoff-entry.csv'):

    bakeoff_test = sst.bakeoff_test_reader(SST_HOME)
    sst_test = sst.test_reader(SST_HOME)
    bakeoff_test['dataset'] = 'bakeoff'
    sst_test['dataset'] = 'sst3'
    df = pd.concat((bakeoff_test, sst_test))

    df['prediction'] = df['sentence'].apply(predict_one_func)

    df.to_csv(output_filename, index=None)

Thus, for example, the following will create a bake-off entry based on `predict_one_softmax`:

In [46]:
# create_bakeoff_submission(predict_one_softmax)

This creates a file `cs224u-sentiment-bakeoff-entry.csv` in the current directory. That file should be uploaded as-is. Please do not change its name.

Only one upload per team is permitted, and you should do no tuning of your system based on what you see in our bakeoff prediction file – you should not study that file in anyway, beyond perhaps checking that it contains what you expected it to contain. The upload function will do some additional checking to ensure that your file is well-formed.

People who enter will receive the additional homework point, and people whose systems achieve the top score will receive an additional 0.5 points. We will test the top-performing systems ourselves, and only systems for which we can reproduce the reported results will win the extra 0.5 points.

Late entries will be accepted, but they cannot earn the extra 0.5 points.